In [6]:
import json
import glob
import PIL
import cv2 as cv
import matplotlib.pyplot as plt

In [7]:
def transform_points(points, image, baseline):
    #top left : 48.89832,2.26693
    # bottom right : 48.81532, 2.42892
    
    #12553px = 15.02km
    #12553px = 15020m
    #1px = 1.19
    
    lower_b = -0.12*(image.shape[0]/image.shape[1])
    for i in range(5):
        points["x"][i] = int(round((points["x"][i]/lower_b)*image.shape[0]))
        points["y"][i] = int(round(((points["y"][i]/0.12)*image.shape[1])))
        
    diff_y = 2.42892-2.26693
    diff_x = 48.89832 - 48.81532
    
    for i in range(5):
        points["y_"][i] = int(round(((points["y_"][i]-2.26693)/diff_y)*baseline.shape[1]))
        points["x_"][i] = int(round((1-(points["x_"][i]-48.81532)/diff_x)*baseline.shape[0]))
    
    temp = points["y_"]
    points["y_"] = points["x_"]
    points["x_"] = temp
    
    temp = points["y"]
    points["y"] = points["x"]
    points["x"] = temp
    

    
    points["image_height"] = image.shape[0]
    points["image_width"] = image.shape[1]
    '''
    for x,y in zip(points["x_"], points["y_"]):
        baseline = cv.circle(baseline, (x,y), 50,(255,0,0),-1)
    plt.imsave("baseline_with_points.png", baseline)
    '''

In [8]:
baseline = cv.imread("baseline.png")
for point in glob.glob("points/*.json"):
    with open(point) as json_file:
        json_data = json.load(json_file)
        if not point[7:-5] == "btv1b53085557h":
            the_map = cv.imread(f"points_full_predictions/{point[7:-5]}.png")
            transform_points(json_data, the_map, baseline)
            with open(f"points_transformed/{point[7:-5]}.json", "w") as transformed_json_file:
                json.dump(json_data, transformed_json_file)